# Import

In [3]:
import csv
import os
import sys

import numpy as np
import pandas as pd
from nltk import word_tokenize
from nltk.corpus import stopwords
from stop_words import get_stop_words
from tqdm import tqdm
import nltk

## Add configuration file

In [4]:
sys.path.append("/home/jovyan/core/config/")
sys.path.append("/home/jovyan/core/util/")

In [5]:
from ALL import config 
from util import *

## Set condition

In [4]:
tqdm.pandas()
pd.set_option("display.max_columns", 100)
pd.set_option("display.max_rows", 50)

# Read data

In [5]:
newsgroups_df = pd.read_csv("../../DataShaping/data/20News/master.csv", index_col=0)

In [6]:
with open("../../DataShaping/data/20News/class.csv", mode="r") as f:
    reader = csv.reader(f)
    class_labels = [label for label in reader]

# Word tokenize

In [7]:
newsgroups_df["words"] = newsgroups_df.text.progress_apply(word_tokenize)

100%|██████████| 18846/18846 [00:47<00:00, 395.77it/s]


# Remove long text

In [8]:
newsgroups_df["words_length"] = newsgroups_df.words.progress_apply(lambda x: len(x))

100%|██████████| 18846/18846 [00:00<00:00, 521809.92it/s]


In [9]:
index = newsgroups_df[
    newsgroups_df["words_length"] < newsgroups_df["words_length"].quantile(0.996)
].index

In [10]:
newsgroups_df = newsgroups_df.loc[index]

# Remove stopwords

In [11]:
stop_words_add = ["would", "could", "should"]
stop_char = ["==", "--", "\'s", "''", "n't", "``","..", "...", "....",".....", "......", "'m", "'ve","'re", "'d", "'ll", "", "-+", "+-", "_/", "||", "__", "/|", "//"]
stop_words = set(stopwords.words("english") + get_stop_words("english") + stop_words_add + stop_char)

In [12]:
#     一文字以下の単語とstop_word, stop_charを削除
newsgroups_df["words_nonstop"] = newsgroups_df.words.progress_apply(
    lambda words: [
        word for word in words if word.lower() not in stop_words if len(word) > 1 
    ]
)

100%|██████████| 18770/18770 [00:01<00:00, 13974.30it/s]


In [13]:
newsgroups_df.words = newsgroups_df.words.progress_apply(lambda words: " ".join(words))
newsgroups_df.words_nonstop = newsgroups_df.words_nonstop.progress_apply(
    lambda words: " ".join(words)
)

100%|██████████| 18770/18770 [00:00<00:00, 168613.70it/s]


In [14]:
file_path = "../data/20News"
os.makedirs(file_path, exist_ok=True)
with open(f"{file_path}/class.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(class_labels)
newsgroups_df.to_csv(f"{file_path}/master.csv")